In [ ]:
import gurobipy as gp
from gurobipy import GRB 
# Network Structure
flow = {
(0, 1): (0, 2),
(0, 2): (1, 8),
(1, 2): (0, 1),
(1, 3): (1, 8),
(2, 3): (0, 1),
(3, 0): (0, float('inf'))
}
# Sets
E = set(flow.keys()) # Arc set
T = 3
V = set(range(T+1)) # Node set
# Parameters
λ = {a: flow[a][0] for a in E} # Lower bounds
k = {a: flow[a][1] for a in E} # Upper bounds

model = gp.Model('Maximum Flow Problem')
x = model.addVars((a for a in E), vtype = GRB.CONTINUOUS, name = "quantity of flow")
model.setObjective(gp.quicksum(x[a] for a in E if a[0] == T and a[1] == 0), GRB.MAXIMIZE)
model.addConstrs((x[a] >= λ[a] for a in E), name="min flow")
model.addConstrs((x[a] <= k[a] for a in E), name="max flow")
model.addConstrs((gp.quicksum(x[a] for a in E if a[1] == j) - gp.quicksum(x[a] for a in E if a[0] == j) == 0 for j in V), name="flow balance")
model.optimize()
print(f'\nThe optimal flow of the network is {model.objVal}')
for a in E:
    print(f'The optimal flow from node {a[0]} to {a[1]} is {x[a].X}')

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (linux64 - "Linux Mint 22.1")

CPU model: 13th Gen Intel(R) Core(TM) i7-1355U, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 16 rows, 6 columns and 24 nonzeros
Model fingerprint: 0x7377df92
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+100]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 16 rows and 6 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.02 seconds (0.00 work units)
Optimal objective  3.000000000e+00

The optimal flow of the network is 3.0
The optimal flow from node 0 to 1 is 2.0
Th

In [ ]:
network = {(1,4):(2,15),(2,1):(1,10),(2,3):(0,10),(3,4):(1,5),(2,6):(6,10),(3,5):(4,10), (4,7):(5,10),(5,6):(2,20),(5,7):(7,15),(6,8):(8,10),(7,8):(9,15)}
# Sets
E = set(network.keys()) # Arc set
V = set(range(1, 9)) # Node set
# Parameters
b = {1:10, 2:20, 3:0, 4:-5, 5:0, 6:0, 7:-15, 8:-10} # Supply/demand
c = {a: network[a][0] for a in E} # Cost on arcs
k = {a: network[a][1] for a in E} # max flow bounds
assert sum(b[j] for j in V if b[j] > 0) + sum(b[j] for j in V if b[j] < 0) == 0, "The sum of supply does not equal the sum of demand!"

model = gp.Model("Minimum Cost Flow Problem")
x = model.addVars((a for a in E), lb=0, vtype = GRB.CONTINUOUS, name = "quantity of flow")
model.setObjective(gp.quicksum(x[a] * c[a] for a in E), GRB.MINIMIZE)
model.addConstrs((x[a] <= k[a] for a in E), name="max flow")
model.addConstrs((gp.quicksum(x[a] for a in E if a[0] == j) - gp.quicksum(x[a] for a in E if a[1] == j) == b[j] for j in V), name="demand fulfillment")
model.optimize()
print(f'\nThe optimal cost of the network is {model.objVal}')
for a in E: 
    print(f'The optimal flow from node {a[0]} to {a[1]} is {x[a].X}')

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (linux64 - "Linux Mint 22.1")

CPU model: 13th Gen Intel(R) Core(TM) i7-1355U, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 19 rows, 11 columns and 33 nonzeros
Model fingerprint: 0x79e68bf1
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 9e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+00, 2e+01]
Presolve removed 19 rows and 11 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.7000000e+02   0.000000e+00   2.300000e+01      0s
Extra simplex iterations after uncrush: 1
       1    2.7000000e+02   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.02 seconds (0.00 work units)
Optimal objective  2.700000000e+02

The optimal cost of the network is 270.0
The optimal flow from node 3 to 4 is 5.0
T